### Intro to Dataframes

In [ ]:
import pandas as pd

df = pd.DataFrame([[1,2,3],[4,5,6],[7,8,9]], columns=['A','B','C'], index=['x','y','z'])

In [ ]:
df.describe()

### Loading in Dataframes from Files


In [ ]:
coffee = pd.read_csv("./warmup-data/coffee.csv")

results = pd.read_parquet("./data/results.parquet")

olympics = pd.read_excel("./data/olympics-data.xlsx")

bios = pd.read_csv("./data/bios.csv")


### Accessing data

In [ ]:
coffee.sample(10)

In [ ]:
coffee.loc[0:6,["Day","Units Sold"]]

In [ ]:
coffee.iloc[0:6,[0,2]] # Only using index

In [ ]:
coffee.loc[1:4,"Unit Solds"] = 10

coffee.head()

In [ ]:
coffee.at[0,"Unit Solds"]=100

In [ ]:
coffee.sort_values("Unit Solds",ascending=True)

coffee.head()

In [ ]:
for idx , row in coffee.iterrows():
    print(idx)
    print(row)
    print("\n")

### Filtering data

In [ ]:
bios.loc[bios['height_cm']>215,["name","height_cm"]]

In [ ]:
bios[(bios["height_cm"]>215)][["name","height_cm"]]

In [ ]:
bios[(bios["height_cm"]>215) & (bios["born_country"]=="USA")]

In [ ]:
bios[bios["name"].str.contains("Hristo",case=False)]

In [ ]:
# Top 5 tallest overall
bios.nlargest(5, 'height_cm')[['name', 'height_cm', 'born_country']]  # top 5 tallest rows


In [ ]:
# Tallest person in each country (per-group max)
bios[bios['height_cm'].eq(bios.groupby('born_country')['height_cm'].transform('max'))][['name', 'born_country', 'height_cm']]  # tallest per country

In [ ]:
# Filter by a set of countries
bios[bios['born_country'].isin(['USA', 'Germany', 'Bulgaria'])][['name', 'born_country', 'height_cm']]  # only selected countries


In [ ]:
# Exclude certain countries
bios[~bios['born_country'].isin({'USA', 'Canada'})][['name', 'born_country', 'height_cm']].head(10)  # not USA/Canada


In [ ]:
# Names that start with A or H (case-insensitive)
bios[bios['name'].str.contains(r'^(A|H)', case=False, na=False)][['name']].head(10)  # anchors at start


In [ ]:
# Duplicate names (possible homonyms)
bios[bios['name'].duplicated(keep=False)].sort_values('name')[['name', 'born_country', 'height_cm']]  # all duplicates


In [ ]:
# Data-quality check: names containing digits
bios[bios['name'].str.contains(r'\d', na=False)][['name']]

In [ ]:
# Shortest person in each country (per-group min via sort+drop_duplicates)
(bios.sort_values(['born_country', 'height_cm'])
     .drop_duplicates('born_country'))[['name', 'born_country', 'height_cm']]  # shortest per country


In [ ]:
bios.query('born_country== "USA" and height_cm > 200')[['name', 'born_country', 'height_cm']]

### Adding / Removing Columns

In [ ]:
coffee['price'] = 5

import numpy as np

coffee['new_price'] = np.where(coffee['Coffee Type'] == 'Espresso',3.99,4.99)

coffee.drop(columns=['price'], inplace=True)

coffee.head()

In [ ]:
## Pointing to the same memory space
coffe_new = coffee

## New one , not modifying the original
coffe_new = coffee.copy()

In [ ]:
coffee.head()

In [ ]:
coffee['revenue'] = coffee['Units Sold'] * coffee['new_price']

In [ ]:
coffee.rename(columns={'new_price':'price'}, inplace=True)

In [ ]:
bios.head()

In [ ]:
bios['first_name'] = bios['name'].str.split(' ').str[0]


bios['born_data_time']=pd.to_datetime(bios['born_date'])

In [ ]:
bios['height_category'] = bios['height_cm'].apply(lambda x : 'Short' if x < 165 else ('Average' if x<185 else 'Tall'))

In [ ]:
def categorize_athlete(row):
    if row['height_cm'] < 175 and row['weight_kg'] < 70:
        return 'Lightweight'
    elif row['height_cm'] < 185 and row['weight_kg'] <= 80:
        return 'Middleweight'
    else:
        return 'Heavyweight'

bios['category'] = bios.apply(categorize_athlete, axis=1)

### Merging & Concatenating Data

In [ ]:
nocs = pd.read_csv('./data/noc_regions.csv')

In [ ]:
nocs.head()

In [ ]:
bios = pd.merge(bios, nocs, left_on='born_country', right_on='NOC', how='left')

bios.rename(columns={'region':'born_country_full'},inplace=True)

In [ ]:
usa = bios[bios.born_country=='USA'].copy()
gbr = bios[bios.born_country=='GBR'].copy()

new_df = pd.concat([usa,gbr])

new_df.head()

### Handling Null Values

In [ ]:
coffee.loc[[0,1],'Units Sold'] = np.nan

In [ ]:
display(coffee)

In [ ]:
coffee= coffee.fillna(coffee['Units Sold'].mean())

In [ ]:
coffee.dropna(subset=['Units Sold'], inplace=True)

In [ ]:
coffee[coffee['Units Sold'].notna()]

### Aggregating Data

In [ ]:
bios.head()

In [ ]:
bios['born_city'].value_counts()

In [ ]:
bios[bios['born_country']=='USA']['born_region'].value_counts()

In [ ]:
coffee.groupby(['Coffee Type'])['Units Sold'].sum()

In [ ]:
coffee.groupby(['Coffee Type']).agg({'Units Sold':'sum','price':'mean'})

In [ ]:
pivot = coffee.pivot(columns='Coffee Type', index='Day' , values = 'revenue')

pivot

In [ ]:
bios.groupby(bios['born_date'])['name'].count().reset_index()

In [ ]:
# Total, average, min, max height per country
bios.groupby('born_country').agg({
    'height_cm': ['count', 'mean', 'min', 'max', 'std']
})


In [ ]:
bios.groupby('born_country').agg(
    num_people=('name', 'count'),
    avg_height=('height_cm', 'mean'),
    tallest=('height_cm', 'max'),
    shortest=('height_cm', 'min')
)


In [ ]:
# Top 10 countries with highest average height
bios.groupby('born_country')['height_cm'].mean().sort_values(ascending=False).head(10)


In [ ]:
# Ensure born_country is a column
if 'born_country' not in bios.columns:
    bios = bios.reset_index()

# Make sure height_cm is numeric
bios['height_cm'] = pd.to_numeric(bios['height_cm'], errors='coerce')

idx = bios.groupby('born_country')['height_cm'].idxmax()
idx = idx.dropna().astype(int)   # remove groups where max is NaN

out = bios.loc[idx, ['born_country', 'name', 'height_cm']].sort_values('born_country')
print(out)


### Advanced Funcs

In [ ]:
coffee['yesterday_revenue'] = coffee['revenue'].shift(2)

In [ ]:
coffee.head()

In [ ]:
bios.head()

In [ ]:
bios['height_rank'] = bios['height_cm'].rank()

bios.sort_values(['height_cm'],ascending=True)

In [ ]:
latte = coffee[coffee['Coffee Type']=='Latte'].copy()

latte['3day']=latte['Units Sold'].rolling(3).sum()
latte

### Other functions

In [ ]:
results.info()

# Pandas – Advanced Cheatsheet Add‑Ons (New Cells)

Below are **new** examples and notes appended to your notebook (your existing content was not modified).

# 1) Powerful Indexing & Selection

Key patterns beyond basics:

- Use `.loc[row_selector, col_selector]` for **label** selection (inclusive slicing), `.iloc` for **position**.
- `.at` / `.iat` are fast for single-scalar get/set.
- Combine boolean masks, `.isin`, and `.between` for expressive filters.
- Set an index for fast lookups: `df = df.set_index('key', drop=False)` keeps column while indexing.
- MultiIndex lets you slice blocks of data elegantly.

In [ ]:

import pandas as pd
import numpy as np

df = pd.DataFrame({
    "city": ["Berlin","Berlin","Munich","Munich","Hamburg","Hamburg"],
    "year": [2023,2024,2023,2024,2023,2024],
    "sales": [120, 150, 90, 110, 80, 130],
    "profit": [12, 18, 8, 11, 7, 14]
})

# Boolean masks
berlin_2024 = df.loc[(df["city"]=="Berlin") & (df["year"]==2024)]
print("Berlin 2024:\n", berlin_2024, "\n")

# Between / isin
big_years = df.loc[df["year"].between(2023, 2024)]
cities = df.loc[df["city"].isin(["Berlin","Hamburg"])]
print("Years 2023-2024:\n", big_years, "\n")
print("Berlin & Hamburg:\n", cities, "\n")

# Fast scalar set with .at
df2 = df.copy()
df2.at[1, "profit"] = 19
print("Scalar set with .at -> profit row 1 now:", df2.loc[1, "profit"], "\n")

# MultiIndex
mi = df.set_index(["city","year"]).sort_index()
print("MultiIndex head:\n", mi.head(), "\n")

# Slice MultiIndex: all rows for Berlin
print("All Berlin via MultiIndex slice:\n", mi.loc[("Berlin", slice(None))], "\n")


# 2) GroupBy Power Moves

Patterns you'll use a lot:

- Named aggregations: `groupby(...).agg(total=('col','sum'), avg=('col','mean'))`
- Use `.transform` to broadcast group stats back to rows (e.g., z-scores per group).
- `.filter` keeps/discards entire groups based on a condition.
- `value_counts()` inside groups via `groupby('g')['x'].value_counts()`.
- `rank` within groups for top-k selections.

In [ ]:

gdf = df.copy()

# Named aggregations
agg = gdf.groupby("city").agg(
    total_sales=("sales","sum"),
    avg_profit=("profit","mean"),
    count=("sales","size")
)
print("Named agg per city:\n", agg, "\n")

# Transform: z-score of sales per city
gdf["sales_z_in_city"] = gdf.groupby("city")["sales"].transform(
    lambda s: (s - s.mean())/s.std(ddof=0)
)
print("Transform z-score per city:\n", gdf, "\n")

# Filter: keep only cities with total sales > 200
filtered = gdf.groupby("city").filter(lambda d: d["sales"].sum() > 200)
print("Filter groups (total sales > 200):\n", filtered, "\n")

# Value counts within groups
vc = gdf.groupby("city")["year"].value_counts()
print("Counts of year within city:\n", vc, "\n")

# Rank within groups (highest sales = rank 1)
gdf["rank_in_city"] = gdf.groupby("city")["sales"].rank(ascending=False, method="dense")
print("Rank within city:\n", gdf.sort_values(["city","rank_in_city"]), "\n")


# 3) Reshaping: pivot, pivot_table, melt, crosstab

When moving between long ↔ wide formats:

- `pivot(index, columns, values)` purely reshapes (no aggregation).
- `pivot_table(..., aggfunc='sum', fill_value=0)` reshapes **with** aggregation.
- `melt` unpivots wide → long.
- `pd.crosstab(rows, cols, values=..., aggfunc=...)` for contingency/summary tables.

In [ ]:

wide = df.pivot(index="city", columns="year", values="sales")
print("pivot (wide):\n", wide, "\n")

pv = pd.pivot_table(df, index="city", columns="year", values="profit", aggfunc="sum", fill_value=0)
print("pivot_table sum of profit:\n", pv, "\n")

long = wide.reset_index().melt(id_vars="city", var_name="year", value_name="sales")
print("melt back to long:\n", long, "\n")

xt = pd.crosstab(df["city"], df["year"], values=df["sales"], aggfunc="sum")
print("crosstab sum of sales:\n", xt, "\n")


# 4) Merge / Join Mastery

Tips:

- Always know your join keys; use `validate='one_to_one'` or `validate='one_to_many'` to catch mistakes.
- `indicator=True` shows where rows came from (`left_only`, `both`, `right_only`).
- Use `suffixes=('_l','_r')` to disambiguate overlapping columns.
- `merge_asof` is great for time‑aware nearest matches (sorted keys!).

In [ ]:

left = pd.DataFrame({"id":[1,2,3], "city":["Berlin","Munich","Hamburg"]})
right = pd.DataFrame({"id":[1,2,4], "year":[2024,2023,2024]})

inner = left.merge(right, on="id", how="inner", indicator=True, validate="one_to_one")
print("Inner merge with indicator:\n", inner, "\n")

outer = left.merge(right, on="id", how="outer", indicator=True, suffixes=("_l","_r"))
print("Outer merge with suffixes:\n", outer, "\n")

# asof example: align timestamps to last seen event
ts1 = pd.DataFrame({"t": pd.to_datetime(["2025-01-01","2025-01-01 12:00","2025-01-02"]), "price":[10, 11, 12]}).sort_values("t")
ts2 = pd.DataFrame({"t": pd.to_datetime(["2025-01-01 06:00","2025-01-01 18:00","2025-01-02 06:00"])}).sort_values("t")
asof = pd.merge_asof(ts2, ts1, on="t", direction="backward")
print("merge_asof backward:\n", asof, "\n")


# 5) Time Series Essentials

Core operations:

- Parse dates with `pd.to_datetime` and set as index for time‑aware ops.
- `resample('D'/'W'/'M')` to change frequency.
- `rolling(window).mean()` for moving averages; `expanding().sum()` for cumulative.
- `shift` for lags/leads; `diff` for first differences.

In [ ]:

idx = pd.date_range("2025-01-01", periods=7, freq="D")
ts = pd.DataFrame({"date": idx, "value": [5,6,7,12,8,9,15]}).set_index("date")

print("Original:\n", ts, "\n")
print("Weekly sum via resample:\n", ts["value"].resample("W").sum(), "\n")
print("3-day rolling mean:\n", ts["value"].rolling(3).mean(), "\n")
print("Lag 1 with shift:\n", ts["value"].shift(1), "\n")
print("First difference:\n", ts["value"].diff(), "\n")


# 6) Missing Data & Nullable dtypes

Modern Pandas has nullable types like `'Int64'` (note the capital I).

- Detect with `.isna()`/`.notna()`; count via `.isna().sum()`.
- Impute with `fillna(value)` or `fillna(method='ffill'|'bfill')`.
- Interpolate numeric series with `.interpolate()`.
- Use `dropna(subset=[...])` to drop rows if certain columns are NA.

In [ ]:

s = pd.Series([1, None, 3], dtype="Int64")
print("Nullable Int64:\n", s, s.dtype, "\n")

df_na = pd.DataFrame({"a":[1,2,None,4], "b":[10,None,30,40]})
print("NA counts:\n", df_na.isna().sum(), "\n")

print("Fillna with scalar:\n", df_na.fillna(0), "\n")
print("Forward fill:\n", df_na.fillna(method="ffill"), "\n")
print("Interpolate column b:\n", df_na["b"].interpolate(), "\n")

print("Drop rows where a is NA:\n", df_na.dropna(subset=["a"]), "\n")


# 7) String Ops & Categories

Vectorized string tools live under `.str` and are super fast.

- `.str.contains`, `.str.extract` (regex), `.str.split`, and `.explode()` for list‑like cells.
- Convert to categories to save memory and speed up groupby/joins when cardinality is low.

In [ ]:

names = pd.Series(["Ada Lovelace","Grace Hopper","Alan Turing","Linus Torvalds"])
print("Contains 'ing':\n", names.str.contains("ing"), "\n")
print("Extract last name:\n", names.str.extract(r"\s([A-Za-z\-]+)$"), "\n")

tags = pd.Series(["a,b", "a", "b,c", "a,c"])
split_exploded = tags.str.split(",").explode()
print("Exploded tags:\n", split_exploded.value_counts(), "\n")

cat = pd.Series(["small","medium","small","large"], dtype="category")
print("Category dtype:", cat.dtype)


# 8) Performance & Memory Tips

- Inspect memory with `df.info(memory_usage='deep')`.
- Downcast numerics: `pd.to_numeric(..., downcast='unsigned'|'integer'|'float')`.
- Prefer vectorized ops over `apply` in hot loops.
- Use `.query()` / `.eval()` for readable filters on large frames.
- Call `.to_numpy()` when you really need NumPy arrays for speed.

In [ ]:

big = pd.DataFrame({
    "a": pd.to_numeric(np.random.randint(0, 1000, size=10_000), downcast="unsigned"),
    "b": pd.to_numeric(np.random.randn(10_000), downcast="float")
})
# Show info (truncated in some environments)
big.info(memory_usage="deep")

# query / eval examples
res = big.query("a < 10 and b > 0")
res2 = big.eval("c = a * b")
print("query rows:", len(res), "| eval new col head:\n", res2.head(), "\n")


# 9) Method Chaining & pipe

Write tidy, readable pipelines with `.pipe` and line breaks:

```python
(out := (
    df
      .query("year >= 2023")
      .assign(margin=lambda d: d.profit / d.sales)
      .pipe(lambda d: d.sort_values(['city','margin'], ascending=[True, False]))
      .reset_index(drop=True)
))
```

In [ ]:

out = (
    df
    .query("year >= 2023")
    .assign(margin=lambda d: d["profit"] / d["sales"])
    .pipe(lambda d: d.sort_values(["city","margin"], ascending=[True, False]))
    .reset_index(drop=True)
)
print(out, "\n")


# 10) Data Quality Checks

Common checks to add early in your workflow:

- Duplicates: `df.duplicated(subset=[...])` → then `drop_duplicates`.
- Ranges: `df['age'].between(0,120)`.
- Schema: `df.dtypes` vs expected, `set(expected_cols) - set(df)`.
- Equality: `df1.equals(df2)` strictly compares values & dtypes.

In [ ]:

dq = pd.DataFrame({
    "id": [1,2,2,3],
    "age": [25, -5, 40, 130]
})
print("Duplicate mask:\n", dq.duplicated(subset=["id"]), "\n")
print("Valid age mask (0..120):\n", dq["age"].between(0,120), "\n")
dq_clean = dq.drop_duplicates(subset=["id"]).query("age.between(0,120)")
print("Cleaned:\n", dq_clean, "\n")


# 12) MultiIndex Patterns: stack/unstack, swaplevel

- `unstack` pivots the **inner** index level to columns.
- `stack` goes the other way.
- `swaplevel` reorders levels; `sort_index` stabilizes after swaps.

In [6]:

mi2 = df.set_index(["city","year"]).sort_index()
unstacked = mi2["sales"].unstack("year")
print("Unstacked sales:\n", unstacked, "\n")

restacked = unstacked.stack()
print("Restacked equals original values:", np.allclose(restacked.values, mi2["sales"].values), "\n")

swapped = mi2.swaplevel(0,1).sort_index()
print("Swap levels:\n", swapped.head(), "\n")


KeyError: "None of ['city', 'year'] are in the columns"